In [1]:


from roboflow import Roboflow
rf = Roboflow(api_key="TZqHzfk27XoP5J9kVtmo")
project = rf.workspace("akashnepak").project("tennis-ball-detection-jus55")
version = project.version(1)
dataset = version.download("yolov5")
                


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to tennis-ball-detection-1 in yolov5pytorch:: 100%|██████████| 1168/1168 [00:00<00:00, 8967.72it/s]


In [3]:
from ultralytics import YOLO


In [7]:
# Check and fix data.yaml file
import os
import yaml

data_yaml_path = f'{dataset.location}/data.yaml'
print(f"Dataset location: {dataset.location}")
print(f"Data.yaml exists: {os.path.exists(data_yaml_path)}")

# Read the current data.yaml
with open(data_yaml_path, 'r') as f:
    data_config = yaml.safe_load(f)

print("Original data.yaml:")
print(data_config)

# Fix the paths - remove the duplicate folder name
base_path = dataset.location
data_config['train'] = f'{base_path}/train/images'
data_config['val'] = f'{base_path}/valid/images'
data_config['test'] = f'{base_path}/test/images'

print("\nFixed data.yaml:")
print(data_config)

# Write the corrected data.yaml
with open(data_yaml_path, 'w') as f:
    yaml.dump(data_config, f, default_flow_style=False)

print("✅ data.yaml file has been corrected!")

Dataset location: /home/akash-1/TENNIS_ANALYSIS/training/tennis-ball-detection-1
Data.yaml exists: True
Original data.yaml:
{'names': ['tennis ball'], 'nc': 1, 'roboflow': {'license': 'CC BY 4.0', 'project': 'tennis-ball-detection-jus55', 'url': 'https://universe.roboflow.com/akashnepak/tennis-ball-detection-jus55/dataset/1', 'version': 1, 'workspace': 'akashnepak'}, 'test': '../test/images', 'train': 'tennis-ball-detection-1/train/images', 'val': 'tennis-ball-detection-1/valid/images'}

Fixed data.yaml:
{'names': ['tennis ball'], 'nc': 1, 'roboflow': {'license': 'CC BY 4.0', 'project': 'tennis-ball-detection-jus55', 'url': 'https://universe.roboflow.com/akashnepak/tennis-ball-detection-jus55/dataset/1', 'version': 1, 'workspace': 'akashnepak'}, 'test': '/home/akash-1/TENNIS_ANALYSIS/training/tennis-ball-detection-1/test/images', 'train': '/home/akash-1/TENNIS_ANALYSIS/training/tennis-ball-detection-1/train/images', 'val': '/home/akash-1/TENNIS_ANALYSIS/training/tennis-ball-detection-1

In [6]:
!yolo task=detect mode=train model=yolov8n.pt data={dataset.location}/data.yaml epochs=100 imgsz=640 batch=16

100%|███████████████████████████████████████| 6.25M/6.25M [00:08<00:00, 761kB/s]
100%|███████████████████████████████████████| 6.25M/6.25M [00:08<00:00, 761kB/s]
Ultralytics 8.3.161 🚀 Python-3.11.3 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 3806MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/akash-1/TENNIS_ANALYSIS/training/tennis-ball-detection-1/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train

In [ ]:
# Load the trained model and validate
model = YOLO('runs/detect/train/weights/best.pt')

# Validate the model
results = model.val()

# Test prediction on a sample image
import glob
test_images = glob.glob(f'{dataset.location}/test/images/*.jpg')
if test_images:
    sample_image = test_images[0]
    results = model(sample_image)
    results[0].show()  # Display results